In [ ]:
import pathlib

import numpy as np
import pandas as pd
import scipy

import plotly.express as xp
import plotly.graph_objects as go

import idelib
import endaq.ide
import endaq.calc
import endaq.plot.plots
from endaq.plot.plots import gen_map

# from endaq.plot. import gen_map
import endaq.cloud
import endaq.io

# IDE Files with `idelib` & `endaq.ide`

## Open an IDE File

In [ ]:
filename = pathlib.Path("test.IDE")

dataset = idelib.importFile(filename)

## Explore Data

In [ ]:
table = endaq.ide.get_channel_table(dataset)
table

## Get Sensor Data

In [ ]:
accel_chs = endaq.ide.get_channels(dataset, measurement_type=endaq.ide.ACCELERATION, subchannels=False)
accel_chs

In [ ]:
def to_pandas(ch):
    data = ch.getSession().arraySlice()
    t, data = (1000*data[0]).astype("timedelta64[ns]"), data[1:].T
    return pd.DataFrame(data, index=t, columns=[sch.name for sch in ch.subchannels])

df_accel = to_pandas(accel_chs[0])
df_accel = df_accel.loc[np.timedelta64(3990, 's'): np.timedelta64(4000, 's')]
display(df_accel)
fig = xp.line(df_accel, x=df_accel.index, y=df_accel.columns)
fig.show()

# Calculations with `scipy` & `endaq.calc`

In [ ]:
import endaq.calc.filters
import endaq.calc.psd
import endaq.calc.shock
import endaq.calc.integrate

## Filters

In [ ]:
df_accel_hp = endaq.calc.filters.butterworth(df_accel, low_cutoff=1, high_cutoff=None)
fig = xp.line(df_accel_hp, x=df_accel_hp.index, y=df_accel_hp.columns)
fig.show()

In [ ]:
df_accel_lp = endaq.calc.filters.butterworth(df_accel, low_cutoff=None, high_cutoff=100)
fig = xp.line(df_accel_lp, x=df_accel_lp.index, y=df_accel_lp.columns)
fig.show()

## Integration

In [ ]:
dfs_integrate = endaq.calc.integrate.integrals(df_accel, n=2, highpass_cutoff=1.0)

for df in dfs_integrate:
    fig = xp.line(df, x=df.index, y=df.columns)
    fig.show()

## PSD

### Linearly-spaced

In [ ]:
dt = (df_accel.index[-1] - df_accel.index[0]) / (len(df_accel) - 1)
fs = np.timedelta64(1, "s") / dt
df_accel_psd = endaq.calc.psd.scipy_welch(df_accel, nperseg=int(11 * fs))

fig = xp.line(df_accel_psd, x=df_accel_psd.index, y=df_accel_psd.columns, log_x=True, log_y=True)
fig.show()

### Octave-spaced

# Plotting with `plotly` & `endaq.plot`

### Derivatives & Integrals

In [ ]:
dt = (df_accel.index[-1] - df_accel.index[0]) / (len(df_accel) - 1)
fs = np.timedelta64(1, "s") / dt
df_accel_psd = endaq.calc.psd.scipy_welch(df_accel, nperseg=int(11 * fs))

df_vel_psd = endaq.calc.psd.differentiate(df_accel_psd, n=-1)
fig = xp.line(df_vel_psd, x=df_vel_psd.index, y=df_vel_psd.columns, log_x=True, log_y=True)
fig.show()

df_jerk_psd = endaq.calc.psd.differentiate(df_accel_psd, n=1)
fig = xp.line(df_jerk_psd, x=df_jerk_psd.index, y=df_jerk_psd.columns, log_x=True, log_y=True)
fig.show()

### Vibration Criterion (VC) Curves

In [ ]:
dt = (df_accel.index[-1] - df_accel.index[0]) / (len(df_accel) - 1)
fs = np.timedelta64(1, "s") / dt
df_accel_psd = endaq.calc.psd.scipy_welch(df_accel, nperseg=int(11 * fs))
df_accel_vc = endaq.calc.psd.vc_curves(df_accel_psd, fstart=1, octave_bins=3)

fig = xp.line(df_accel_vc, x=df_accel_vc.index, y=df_accel_vc.columns, log_x=True, log_y=True)
fig.show()

## Shock Analysis

In [ ]:
df_accel_pvss = endaq.calc.shock.pseudo_velocity(df_accel, freqs=2 ** np.arange(-3, 13, 0.25), damp=0.05)

fig = xp.line(df_accel_pvss, x=df_accel_pvss.index, y=df_accel_pvss.columns, log_x=True, log_y=True)
fig.show()

## Shock Characterization: Half-Sine-Wave Pulse

In [ ]:
half_sine_params = endaq.calc.shock.half_sine_shock_envelope(df_accel_pvss, damp=0.05)
half_sine_params

## GPS Plot

In [ ]:
gps_data = pd.DataFrame(
    
)

In [ ]:
def get_doc(url):
    r = requests.get(url)
    f = io.BytesIO(r.content)

    doc = idelib.importer.openFile(f, getExitCond=False, quiet=True)
    idelib.importer.readData(doc)
    doc.__dict__
    return doc


def get_channels_table(doc):
    column_strings = [
        "CH #",
        "Sub ID",
        "CH ID",
        "Channel Name",
        "Subchannel Name",
        "Type",
        "Units",
        "Remove Mean",
        #"Length",
    ]

    df_ide = pd.DataFrame(columns=column_strings)

    dicts = list(doc.channels.items())
    for j in range(len(dicts)):
        channel_id = list(dicts[j])[0]
        cur_channel = doc.channels[channel_id]
        for i in cur_channel.subchannels:
            index_loc = i.id / 10 + channel_id

            df_ide.loc[index_loc, column_strings] = [
                j, i.id, channel_id, cur_channel.displayName, i.displayName, i.units[0], i.units[1],
                cur_channel.allowMeanRemoval]

            df_ide.loc[index_loc, "Length"] = len(cur_channel.getSession())

    return df_ide


def get_channel(doc, channel_id):
    Data = doc.channels[channel_id].getSession().arraySlice().T

    names = ["Time (s)"]
    for i in doc.channels[channel_id].subchannels:
        names.append(i.displayName)

    df = pd.DataFrame(data=Data, columns=names)
    df["Time (s)"] = df["Time (s)"] / 1e6

    return df.set_index("Time (s)")


def get_channel_ids(doc):
    return [list(d)[0] for d in list(doc.channels.items())]


def get_sample_rate(df):
    return len(df) / (df.index[-1] - df.index[0])


def get_all_channels(df_ide, doc):
    dfs = []
    df_ide["Frequency (Hz)"] = 0
    df_ide["Length"] = 0
    df_ide["STD"] = 0
    df_ide["Max"] = 0
    df_ide["Min"] = 0
    for i in get_channel_ids(doc):
        df = get_channel(doc, i)
        fs = get_sample_rate(df)
        remove_mean = df_ide.loc[df_ide["CH ID"] == i, "Remove Mean"].iloc[0]
        if remove_mean:
            df -= df.mean()

        for c in df.columns:
            df_ide.loc[df_ide["Subchannel Name"] == c, ['STD', 'Max', 'Min']] = (df[c].std(), df[c].max(), df[c].min())

        dfs.append(df)

        df_ide.loc[df_ide["CH ID"] == i, ["Frequency (Hz)", "Length"]] = (fs, len(df))
        
    df_ide["Name"] = (
            df_ide["CH ID"].astype(str)
            + "."
            + df_ide["Sub ID"].astype(str)
            + ": "
            + df_ide["Subchannel Name"]
            + " ["
            + df_ide["Units"]
            + "]"
    )
    return df_ide, dfs

def get_channel_ids(doc):
    return [list(d)[0] for d in list(doc.channels.items())]

In [ ]:
doc_table, dfs = get_all_channels(get_channels_table(dataset), dataset)

In [ ]:
doc_channel_ids = get_channel_ids(dataset)
right_fig = gen_map(dfs[doc_channel_ids.index(88)], mapbox_access_token="pk.eyJ1IjoiaGFubWFuNDI5IiwiYSI6ImNrbGlmd3o4eTVsYmgycHVpdHBjajJqa3gifQ.D_b7Pt2xW7H_fqqZzqBQHQ")

# endaq.plot.plots.gen_map(dfs
right_fig
# dfs[doc_channel_ids.index(88)]

In [ ]:
xp.line(dfs[1], dfs[1].index, 'Acceleration: X (40g)')